In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
from utils import ui

import flask
from flask_cors import CORS
from flask import jsonify

app = flask.Flask(__name__)
CORS(app)
app.config["DEBUG"] = False

def get_last_result():
    ###############################
    #### load model prediction ####
    ###############################
    result = pd.read_csv('../../data/processed/last_result.csv')
    result = result.loc[result.present_time == result.present_time.max(),
                       ].reset_index(drop=True)

    present_time = result.present_time[0]
    #####################
    ### Model results ###
    #####################
    result['Predicted Scenario'] = result.apply(lambda row : ui.get_str_scenario(row['pred_speed'],
                                                                            row['cos_wind_dir'],
                                                                            row['sin_wind_dir']),
                                           axis=1 )
    result['Predicted Binary'] =result['Predicted Scenario'].map(ui.scenario_to_binary)

    result['pred_speed'] = result['pred_speed'].map(lambda x : str(round(x,1)))
    result['pred_wind_dir'] = result['pred_wind_dir'].map(lambda x : str(round(x,1)))

    result.rename(columns={'pred_wind_dir': 'Predicted Wind Direction', 'pred_speed': 'Predicted Wind Speed'},inplace=True)


    #########################
    #### Numtech results ####
    #########################
    result['numtech_cos_wind_dir'] = np.cos(2 * np.pi * result['numtech_wind_dir'] / 360)
    result['numtech_sin_wind_dir'] = np.sin(2 * np.pi * result['numtech_wind_dir'] / 360)

    result['Numtech Scenario'] = result.apply(lambda row : ui.get_str_scenario(row['numtech_speed'],
                                                                            row['numtech_cos_wind_dir'],
                                                                            row['numtech_sin_wind_dir']),
                                           axis=1 )
    result['Numtech Binary'] =result['Numtech Scenario'].map(ui.scenario_to_binary)

    result.rename(columns={'numtech_wind_dir': 'Numtech Wind Direction', 'numtech_speed': 'Numtech Wind Speed'},inplace=True)

    result.rename(columns={'datetime': 'Date'},inplace=True)

    result = result[['Date','Predicted Binary','Predicted Scenario','Predicted Wind Speed', 'Predicted Wind Direction',
                     'Numtech Binary','Numtech Scenario','Numtech Wind Speed', 'Numtech Wind Direction']]

    result['Day'] = result['Date'].map(lambda x : str(x[0:10]))
    result['Time'] = result['Date'].map(lambda x : str(x[11:13]) + 'h')
    result.drop(columns={'Date'},inplace=True)
    return result

@app.route('/data/last_results/', methods=['GET'])
def home():
    return jsonify(get_last_result().to_dict(orient='records'))
app.run(port='5001')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Aug/2021 16:04:11] "GET /data/last_results/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:04:11] "GET /data/last_results/ HTTP/1.1" 200 -
